In [25]:
import nltk
import numpy as np
import re
from nltk.tokenize import RegexpTokenizer
import collections
import operator

In [22]:
import networkx as nx
import scipy.sparse as sps    
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

In [26]:
def text_rank_keywords(raw_text, n_top_keywors = 10):
    raw_text = re.sub(r'\d+', '', raw_text)
    words = nltk.word_tokenize(raw_text)
    words = [word.lower() for word in words if word[0].isalpha()]


    #words = [w.lower() for w in text]
    wnl = nltk.WordNetLemmatizer()
    words = [wnl.lemmatize(w) for w in words]

    wordList=np.unique(words)
    wordList_filtered = np.array([word for word in wordList if word not in set(nltk.corpus.stopwords.words('english'))])

    tagged = nltk.pos_tag(wordList_filtered)


    def filter_for_tags(tagged, tags=['NN', 'JJ', 'NNP']):
        return [item for item in tagged if item[1] in tags]

    tagged = filter_for_tags(tagged)
    wordList_filtered = np.array(list(map(lambda x : x[0], tagged)))

    window_size = 3
    adj_matr = sps.lil_matrix((len(wordList_filtered), len(wordList_filtered)), dtype="float64")

    for i, word in enumerate(words):
        src = np.where(wordList_filtered == words[i])
        if len(src) > 0:
            for j in np.arange(i+1, np.min([i+window_size, len(words)])):
                dest = np.where(wordList_filtered == words[j])
                if len(dest) > 0:
                    try:
                        adj_matr[src, dest] = adj_matr[src, dest] + 1
                    except:
                        adj_matr[src, dest] = 1

    adj_matr = sps.csr_matrix(adj_matr)
    graph = nx.Graph(adj_matr)

    textrank = nx.pagerank(graph)

    sorted_by_rank = sorted(textrank.items(), key=operator.itemgetter(1))
    keywords = list(map(lambda x : x[0], sorted_by_rank[:-n_top_keywors-1:-1]))
    return wordList_filtered[np.array(keywords)]

In [32]:
from dataset_500 import DataReader
dataset = DataReader().fetch_data()
x, y = dataset.train_data

array(['lopilato', 'buble', 'pop', 'ap', 'concert', 'star', 'actress',
       'buenos', 'ceremony', 'month'],
      dtype='<U14')

In [40]:
s = 0
n = len(x)
for i in range(n):
    ans = text_rank_keywords(x[i])
    s += len(set(map(lambda x : x.lower(), y[i])) & set(ans))
s/n

3.3955555555555557

In [46]:
text_ind = 67
ans = text_rank_keywords(x[text_ind])
set(map(lambda x : x.lower(), y[text_ind])) & set(ans)

{'continued', 'job', 'recovery', 'unemployment'}

In [43]:
ans

array(['lopilato', 'buble', 'pop', 'ap', 'concert', 'star', 'actress',
       'buenos', 'ceremony', 'month'],
      dtype='<U14')

In [44]:
y[text_ind]

['married Argentine',
 'his Grammy',
 'ceremony',
 'kisses',
 'messages',
 'romantically',
 'vocal',
 'Argentine',
 'woman',
 'Buble dated',
 'thanking',
 'wedding ceremony',
 'Canadian',
 'Michael',
 'bouquet',
 'vocal album',
 'Crazy Love',
 'Canadian pop',
 'Argentina AP',
 'mansion',
 'Facebook',
 'rose',
 'Argentine sweetheart',
 'month',
 'registry',
 'South American',
 'after tying',
 'downtown',
 'beautiful',
 'traditional',
 'Grammy',
 'Vancouver',
 'chiffon dress',
 'Grammywinning',
 'family',
 'Lopilato made',
 'North',
 'bride Argentine TV actress',
 'media',
 'Twitter',
 'met',
 'high']